# Semantix Desafio Engenheiro de Dados

## HTTP requests to the NASA Kennedy Space Center WWW server


Fonte oficial do dateset: http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html  

Dados:
* Jul 01 to Jul 31, ASCII format, 20.7 MB gzip compressed, 205.2 MB (ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz).
* Aug 04 to Aug 31, ASCII format, 21.8 MB gzip compressed, 167.8 MB (ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz).  

Sobre o dataset: Esses dois conjuntos de dados possuem todas as requisições HTTP para o servidor da NASA Kennedy
Space Center WWW na Flórida para um período específico.

  
  
  Os logs estão em arquivos ASCII com uma linha por requisição com as seguintes colunas:
* Host fazendo a requisição. Um hostname quando possível, caso contrário o endereço de internet se o nome
não puder ser identificado.
* Timestamp no formato "DIA/MÊS/ANO:HH:MM:SS TIMEZONE"
* Requisição (entre aspas)
* Código do retorno HTTP
* Total de bytes retornados

### Introdução

Para resolução deste desafio está sendo utilizado o Google Golab que permite rodar o Spark.  
A linguagem utilizada será Python.


### Preparando o ambiente e os dados

Primeiro é feita a instalação do Java, do Spark com o Hadoop, do findSpark (utilizado para encontrar a instalação do Spark na máquina) e do PySpark (API de Spark em Python).  
Segundo, são configuradas as variáveis de ambiente.  
Terceiro é iniciado uma sessão do Spark.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
!tar xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark

     |████████████████████████████████| 215.6MB 66kB/s 
     |████████████████████████████████| 204kB 39.1MB/s 


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Baixando os arquivos

In [0]:
!wget -q ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz
!wget -q ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz

Importando dos dados dos arquivos

In [0]:
sc = spark.sparkContext

dadosJul = sc.textFile("NASA_access_log_Jul95.gz")
dadosAgo = sc.textFile("NASA_access_log_Aug95.gz")


In [6]:
dadosJul.take(10)

['199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245',
 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085',
 'burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0',
 '199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179',
 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0',
 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0',
 '205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985',
 'd104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '129.94.144.152 - - [01/Jul/

In [7]:
dadosAgo.take(10)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0',
 'ix-esc-ca2-07.ix.netcom.com - - [01/Aug/1995:00:00:09 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1713',
 'uplherc.upl.com - - [01/Aug/1995:00:00:10 -0400] "GET /images/WORLD-logosmall.gif HTTP/1.0" 304 0',
 'slppp6.intermind.net - - [01/Aug/1995:00:00:10 -0400] "GET /history/skylab/skylab.html HTTP/1.0" 200 1687',
 'piweba4y.prodigy.com - - [01/Aug/1995:00:00:10 -0400] "GET /images/launchmedium.gif HTTP/1.0" 200 11853',
 'slppp6.intermind.net - - [01/Aug/1995:00:00:11 -0400] "GET

In [0]:
dados = sc.union([dadosJul, dadosAgo])

In [9]:
dados.count()

3461613

Verifica se os arquivos contém linhas fora do padrão. Que não contém as colunas descritas no enunciado.

In [0]:
linhasForaPadrao = dados.map(lambda linha: linha.split(" ")).filter(lambda lista: len(lista) < 5)

In [11]:
linhasForaPadrao.take(10)

[['alyssa.p']]

A linha mostrada acima, será retirada dos dados porque não está no padrão do restante do dataset.

In [0]:
dados = dados.filter(lambda linha: linha != "alyssa.p")

In [13]:
dados.count()

3461612

### Questões

1. Número de hosts únicos.



In [14]:
dados.map(lambda linha: linha.split(" - -")) \
  .map(lambda lista: lista[0]) \
  .distinct() \
  .count()


137978

2. O total de erros 404.


In [15]:
requisicoes404 = dados.map(lambda linha: linha.split(" ")) \
  .filter(lambda lista: lista[-2] == "404")

requisicoes404.count()

20901

3. Os 5 URLs que mais causaram erro 404.


In [16]:
host404 = requisicoes404.map(lambda lista: lista[0]) \
  .map(lambda host: (host, 1)) \
  .reduceByKey(lambda a, b: a + b) \
  .sortBy(lambda x: x[1],False)

host404.take(5)

[('hoohoo.ncsa.uiuc.edu', 251),
 ('piweba3y.prodigy.com', 157),
 ('jbiagioni.npt.nuwc.navy.mil', 132),
 ('piweba1y.prodigy.com', 114),
 ('www-d4.proxy.aol.com', 91)]

4. Quantidade de erros 404 por dia.


In [17]:
dia404 = requisicoes404.map(lambda lista: lista[3].replace('[', '')) \
  .map(lambda dia: (dia.split(":",1))[0]) \
  .map(lambda dia: (dia, 1)) \
  .reduceByKey(lambda a, b: a + b)

dia404.sortBy(lambda x: x[0]) \
  .sortBy(lambda x: (x[0].split("/"))[1],False) \
  .collect()

[('01/Jul/1995', 316),
 ('02/Jul/1995', 291),
 ('03/Jul/1995', 474),
 ('04/Jul/1995', 359),
 ('05/Jul/1995', 497),
 ('06/Jul/1995', 640),
 ('07/Jul/1995', 570),
 ('08/Jul/1995', 302),
 ('09/Jul/1995', 348),
 ('10/Jul/1995', 398),
 ('11/Jul/1995', 471),
 ('12/Jul/1995', 471),
 ('13/Jul/1995', 532),
 ('14/Jul/1995', 413),
 ('15/Jul/1995', 254),
 ('16/Jul/1995', 257),
 ('17/Jul/1995', 406),
 ('18/Jul/1995', 465),
 ('19/Jul/1995', 639),
 ('20/Jul/1995', 428),
 ('21/Jul/1995', 334),
 ('22/Jul/1995', 192),
 ('23/Jul/1995', 233),
 ('24/Jul/1995', 328),
 ('25/Jul/1995', 461),
 ('26/Jul/1995', 336),
 ('27/Jul/1995', 336),
 ('28/Jul/1995', 94),
 ('01/Aug/1995', 243),
 ('03/Aug/1995', 304),
 ('04/Aug/1995', 346),
 ('05/Aug/1995', 236),
 ('06/Aug/1995', 373),
 ('07/Aug/1995', 537),
 ('08/Aug/1995', 391),
 ('09/Aug/1995', 279),
 ('10/Aug/1995', 315),
 ('11/Aug/1995', 263),
 ('12/Aug/1995', 196),
 ('13/Aug/1995', 216),
 ('14/Aug/1995', 287),
 ('15/Aug/1995', 327),
 ('16/Aug/1995', 259),
 ('17/Aug/19

5. O total de bytes retornados.

In [18]:
dados.map(lambda linha: linha.split(" ")) \
  .filter(lambda lista: lista[-1].isnumeric()) \
  .map(lambda lista: int(lista[-1])) \
  .sum()

65524314915